In [50]:
import pyodbc as dbc
import pandas as pd
import sqlalchemy as sa
import os

DB1_NAME = 'opsmdd_CRDH_DEV.mdb'
DB2_NAME = 'opsmdd_9_04_24_Load.mdb'

In [51]:
def get_driver() -> str:
    driver_names = [x for x in dbc.drivers() if "*.mdb" in x]
    if driver_names:
        return driver_names[0]

In [52]:
# Function to connect to an Access Database (.mdb)
def create_eng(db_path:str) -> sa.Engine:
    conn_str = r'DRIVER={}; DBQ={};'.format(get_driver(), db_path)
    connection_url = sa.engine.URL.create(
        "access+pyodbc",
        query={"odbc_connect": conn_str}
    )
    return sa.create_engine(connection_url)

In [53]:
# Function to read a table into a pandas DataFrame
def read_table(connection:dbc.Connection, table_name:list) -> pd.DataFrame:
    query = f"SELECT * FROM [{table_name}]"
    return pd.read_sql(query, connection)

In [54]:
# Get the current working directory
current_dir = os.getcwd()
# Build full paths for the database files
db1_path = os.path.join(current_dir, DB1_NAME)
db2_path = os.path.join(current_dir, DB2_NAME)

In [55]:
# Create sql engine
db1_engine = create_eng(db1_path)
db2_engine = create_eng(db2_path)

In [56]:
# Read Table
df1 = pd.read_sql('vars', db1_engine)
df2 = pd.read_sql('vars', db2_engine)

In [57]:
# Set display to 26 to see all columns
pd.set_option('display.max_columns', 51)
pd.set_option('display.max_rows', 10)

In [105]:
df1.set_index('f_ptid', inplace=True)
df2.set_index('f_ptid', inplace=True)

In [106]:
# df1['source'] = 'db1'
# df2['source'] = 'db2'

In [107]:
# df1.head(3)

In [108]:
# df2.head(3)

In [109]:
merged = pd.merge(df1, df2, on='f_ptid', how='outer', indicator=True)

In [111]:
merged.head(3)

,f_flag_x,f_user_x,f_brief_x,f_gid_x,f_vco_x,f_dtype_x,f_precs_x,f_addr_x,f_ndim_x,f_sys_x,f_unit_x,f_value_x,f_pred_x,f_bflag_x,f_dim1_x,f_dim2_x,f_dim3_x,f_bound_x,f_size_x,f_updat_x,f_time_x,f_ldes_x,f_sflag_x,f_tflag_x,rid_x,f_flag_y,f_user_y,f_brief_y,f_gid_y,f_vco_y,f_dtype_y,f_precs_y,f_addr_y,f_ndim_y,f_sys_y,f_unit_y,f_value_y,f_pred_y,f_bflag_y,f_dim1_y,f_dim2_y,f_dim3_y,f_bound_y,f_size_y,f_updat_y,f_time_y,f_ldes_y,f_sflag_y,f_tflag_y,rid_y,_merge
f_ptid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACWSCNET,o,None,CW BASIN NET MASS IN/OUT lb,2.0,v,r,4.0,83996.0,0.0,None,None,None,global02,,0.0,0.0,0.0,83996.0,4.0,NaN,1.584468e+09,None,None,None,149323.0,o,None,CW BASIN NET MASS IN/OUT lb,2.0,v,r,4.0,83996.0,0.0,None,None,None,global02,,0.0,0.0,0.0,83996.0,4.0,NaN,1.584468e+09,None,None,None,149323.0,both
AGEN2AG1,o,None,Phase A Current E-RLY-11/G1 (E-COMP-,30.0,v,r,4.0,3636.0,0.0,None,AMPS,None,global30,,0.0,0.0,0.0,3636.0,4.0,NaN,1.685461e+09,None,None,None,207699.0,o,None,Phase A Current E-RLY-11/G1 (E-COMP-,30.0,v,r,4.0,3636.0,0.0,None,AMPS,None,global30,,0.0,0.0,0.0,3636.0,4.0,NaN,1.685461e+09,None,None,None,207699.0,both
AGEN2AG2,o,None,Phase A Current E-RLY-11/G2 (E-COMP-,30.0,v,r,4.0,3640.0,0.0,None,AMPS,None,global30,,0.0,0.0,0.0,3640.0,4.0,NaN,1.685461e+09,None,None,None,207700.0,o,None,Phase A Current E-RLY-11/G2 (E-COMP-,30.0,v,r,4.0,3640.0,0.0,None,AMPS,None,global30,,0.0,0.0,0.0,3640.0,4.0,NaN,1.685461e+09,None,None,None,207700.0,both


In [112]:
merged['_merge'].value_counts()

_merge
both          171699
left_only        294
right_only       114
Name: count, dtype: int64

In [178]:
new_in_db1_filter = (merged['_merge'] == 'left_only')
new_in_db2_filter = (merged['_merge'] == 'right_only')

In [179]:
db1_only = merged.loc[new_in_db1_filter]
db2_only = merged.loc[new_in_db2_filter]

In [180]:
filter_x = (db1_only.columns.str.endswith('_x'))
filter_y = (db2_only.columns.str.endswith('_y'))

In [181]:
db1_only = db1_only.loc[:,filter_x]
db2_only = db2_only.loc[:,filter_y]

In [185]:
# new_df = db1_only.rename(columns=lambda s: s.replace("_x", ""))
# new_df = db1_only.rename(columns={'ConvertedComp': 'SalaryUSD'}, inplace=True)
db1_only.columns = [x.replace("_x", "") for x in db1_only.columns]
db2_only.columns = [x.replace("_y", "") for x in db2_only.columns]

In [192]:
# db1_only.sort_values(by='rid', ascending=True)
# df1.sort_values(by='rid', ascending=True)

,f_flag,f_user,f_brief,f_gid,f_vco,f_dtype,f_precs,f_addr,f_ndim,f_sys,f_unit,f_value,f_pred,f_bflag,f_dim1,f_dim2,f_dim3,f_bound,f_size,f_updat,f_time,f_ldes,f_sflag,f_tflag,rid
f_ptid,,,,,,,,,,,,,,,,,,,,,,,,,
gxxm004,o,ops,SPARE,2.0,v,l,1,123764.0,0,rn,None,None,kmot_rst,b,0,0.0,0.0,242.0,1.0,980519.0,1.807340e+05,SPARE \n,None,None,-155812
xsrqb02p,o,ops,* RSCS RED/AMBE,2.0,v,l,1,36940.0,0,rn,None,None,lrscyel,b,0,0.0,0.0,1.0,1.0,980518.0,1.758290e+05,* RSCS RED/AMBE \n,None,None,-95737
cgssuatk,o,ops,HXFR CNST: VESSELS,4.0,c,r,4,18668.0,1,rn,None,None,global04,None,3,0.0,0.0,18668.0,12.0,980519.0,1.353560e+05,HXFR CNST: VESSELS \n,None,None,-82587
ufdm004b,o,ops,clf value: REDUCED H,2.0,v,r,4,253792.0,0,RN,None,None,global02,None,0,0.0,0.0,253792.0,4.0,981114.0,1.717230e+05,clf value: REDUCED HEAD RX BLDG FDR SUMP\n,None,None,-78288
imlnis6d,o,ops,APRM FLOW UNIT D FAIL,2.0,v,i,1,248568.0,0,rn,None,None,imlnis6,b,0,0.0,0.0,3.0,1.0,981029.0,2.338190e+05,APRM FLOW UNIT D FAIL\n,None,None,-74996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCUJCRDPRSS,o,None,"BALL CHK VLV LOGICAL,L",16.0,v,l,1,53382.0,1,None,None,None,global16,,185,0.0,0.0,53382.0,185.0,NaN,1.726864e+09,"BALL CHK VLV LOGICAL,L\r\n",None,None,209844
HCUJRDH121,o,None,HCU DIRECTION VLV CRD-V-121,16.0,v,r,4,736260.0,1,None,None,None,global16,,185,0.0,0.0,736260.0,740.0,NaN,1.727098e+09,HCU DIRECTION VLV CRD-V-121\r\n,None,None,209845
HCUJRDH123,o,None,HCU DIRECTION VLV CRD-V-123,16.0,v,r,4,737000.0,1,None,None,None,global16,,185,0.0,0.0,737000.0,740.0,NaN,1.727098e+09,HCU DIRECTION VLV CRD-V-123\r\n,None,None,209846


In [195]:
output_file = os.path.join(current_dir, 'diffreport.txt')
with open(output_file, 'a') as f:
    if not db1_only.empty:
        f.write(f"New entries in table 'vars' found only in {db1_name}:\n")
        f.write(db1_only.to_string(index=False))
        f.write("\n\n")
    else:
        f.write(f"No new entries in table 'vars' found only in {db1_only}.\n\n")

NameError: name 'output_file' is not defined